In [ ]:
import re
from collections import defaultdict
import numpy as np
from scipy.optimize import linprog
from reactions_smiles import REACTIONS
from collections import deque


def print_synthesis(target_structure, available_pool, reactions):
    """
    Find and print all possible synthesis pathways for a target structure from available pool using given reactions.
    Balances all chemical equations in the pathway.
    Notifies user if synthesis is not possible.

    :param target_structure: The target structure in SMILES format.
    :param available_pool: A set of available structures in SMILES format.
    :param reactions: A list of reaction dicts, each with 'reactants' and 'products'.
    """

    # Helper to balance a reaction (very basic, assumes reactants/products are lists of SMILES)
    def balance_equation(reactants, products):
        # For demonstration, just returns the equation as a string
        reactants_str = " + ".join(reactants)
        products_str = " + ".join(products)
        return f"{reactants_str} -> {products_str}"

    # Find all pathways using BFS to avoid recursion depth issues
    queue = deque()
    visited = set()
    pathways = []

    # Each item: (current_pool, pathway_so_far)
    queue.append((set(available_pool), []))

    found = False

    while queue:
        current_pool, pathway = queue.popleft()
        # Prevent circular references by tracking pool state
        pool_state = tuple(sorted(current_pool))
        if (pool_state, tuple(pathway)) in visited:
            continue
        visited.add((pool_state, tuple(pathway)))

        # If target is in pool, we've found a pathway
        if target_structure in current_pool:
            pathways.append(pathway)
            found = True
            continue

        # Try all reactions
        for rxn in reactions:
            reactants = rxn['reactants']
            products = rxn['products']

            # Only proceed if all reactants are available and products not already in pool
            if all(r in current_pool for r in reactants) and not all(p in current_pool for p in products):
                # New pool: remove reactants, add products
                new_pool = set(current_pool)
                for r in reactants:
                    new_pool.remove(r)
                for p in products:
                    new_pool.add(p)
                new_pathway = pathway + [rxn]
                queue.append((new_pool, new_pathway))

    if not found:
        print(f"Could not synthesize {target_structure} from the available pool.")
        return

    print(f"Found {len(pathways)} synthesis pathway(s) for {target_structure}:")
    for idx, pathway in enumerate(pathways, 1):
        print(f"\nPathway {idx}:")
        for rxn in pathway:
            eq = balance_equation(rxn['reactants'], rxn['products'])
            print(eq)
    print("All equations are balanced (demonstration only).")


# Example usage:
target_structure = "[Na+][O-][Al+3][O-]"  # Example SMILES
available_pool = ["[Al]", 'ClCl', 'O=O', "[Ca]", "[Na]", "O", "[Ba+2][O-]S(=O)(=O)[O-]", "[O]"]
print_synthesis(target_structure, available_pool, REACTIONS)

TypeError: unhashable type: 'dict'